# Chatbot Hospitality Industry
<hr>

In [8]:
# Import

import transformers
#from nltk.corpus import wordnet

import speech_recognition as sr
from gtts import gTTS

import os
import re

In [9]:
# var

# Variable for the towel data 
towel_req = [[0,0],         #      , room number
             [0,0],         #  flag, count bath towel 
             [0,0],         #  flag, count face towel
             [0,0]]         #  flag, count washcloth

key_towel = '.*\\bi need some towels\\b.*|.*\\bi need some towel\\b.*' #"I need some towels"

In [10]:
### 

<hr>

In [11]:
# gTTS - Convert Text to speech

def text_to_speech(text):
    print("Virtual Assistant: ", text)
    speaker = gTTS(text=text, lang="en", slow=False)
    speaker.save("res.mp3")
    os.system("afplay res.mp3") 
    os.remove("res.mp3")
    

In [12]:
# Speech Recognition, using speech_recognition

def speech_to_text():
    text = ""
    recognizer = sr.Recognizer()
    with sr.Microphone() as mic:
        print("listening now")
        audio = recognizer.listen(mic)
    try:
        text = recognizer.recognize_google(audio)
        print("Human:",text)
    except:
        print("Human: Error")
        text_to_speech('i am sorry i did not get that.')
    return text.lower()

<hr>

In [13]:
# Number of Towels


def numberofTowels(type_of_towel):
    yes = '.*\\byes\\b.*|.*\\byep\\b.*|.*\\bcorrect\\b.*'
    clean_input = 0     

    vaReply = "how many " + type_of_towel + " towels would you like?"
    text_to_speech(vaReply)
    
    while (True):
    
        # input 
        human_input = speech_to_text()
        # only the numbers
        try:
            clean_input = int(re.sub('\D', '', human_input) )
        except:
            vaReply = "I a sorry i did not get that.  How many " + type_of_towel + " towels would you like?"
            text_to_speech(vaReply)
            print('except')
             
        if (clean_input <= 20) & (clean_input > 0): 
            return clean_input
            
        elif clean_input > 20:
            vaReply = 'i am sorry the max towels we can send you is 20.  We will send you 20.'    
            text_to_speech(vaReply)
            return 20
        else:
            print('is this another converstation')
    
        

<hr>

In [14]:
# I need some towels 

def need_towels():
    yes = '.*\\byes\\b.*|.*\\byep\\b.*|.*\\bcorrect\\b.*'
    no =  '.*\\bno\\b.*|.*\\bnegative\\b.*|.*\\bnope\\b.*'
    
    keyRoomNumber = False
    flag = True
    
    
    # Loop for towel input 
    while(flag == True):
        
        if towel_req[1][0] == 0 & towel_req[2][0] == 0 & towel_req[3][0] == 0: 
            exit = False
            
            # output
            vaReply = "what type of towels would you like: bath towels, hand towels, or washcloths"
            text_to_speech(vaReply)
            
            # loop for towel 
            while exit == False:
                
            ## #### Input -- type of towel
                human_input = speech_to_text()
                
                
            ## #### ## #### WHAT TYPE OF TOWEL
            
                if re.search('.*\\bbath\\b.*',human_input):
                    towel_req[1][0] = 1 
                    flag = 1
                    # number of towels
                    towel_req[1][1] = numberofTowels('bath') 
                    
                elif re.search('.*\\bhand\\b.*|.*\\bhands\\b.*',human_input):
                    towel_req[2][0] = 1 
                    flag = 1
                    # number of towels
                    towel_req[2][1] = numberofTowels('hand') 
                    
                elif re.search('.*\\bwashcloth\\b.*',human_input):
                    towel_req[3][0] = 1 
                    flag = 1
                    # number of towels
                    towel_req[3][1] = numberofTowels('washcloth') 
                
                else:
                    print('ELSE need some towels ')
                    #while 
                        #smart-chat responce
                        # maybe if smart chatbot responce cosi is > ___ threshhold 
                    vaReply = "what type of towels would you like: bath towel, hand towel, or washcloths"
                    text_to_speech(vaReply) 
                    
                    
            ### ### ### DO YOU NEED ANOTHER TYPE OF TOWEL ### ### ### ###


                if towel_req[1][0] == 1 | towel_req[2][0] == 1 | towel_req[3][0] == 1: 
                    
                    while (exit == False):
                        
                        # output 
                        vaReply = "would you like any additional type of towel?"
                        text_to_speech(vaReply)
                        # input 
                        human_input = speech_to_text()
                        
                        if re.search(yes,human_input):
                            #vaReply = "what type of towels would you like: bath towel, hand towel, or washcloths"
                            text_to_speech(vaReply)
                        elif re.search(no,human_input):
                            # flag to exit towel type loop
                            exit = True
                        else:
                            vaReply = "I am sorry i did not understand that. would you like any additional type of towel?"
                            text_to_speech(vaReply)
            ### ###### ###          
                    
            ### ###### ### ROOM NUMBER ###### ### ###### ###  
            
            # keep room number and change towels
            towel_req[0][0] = roomNumber()
            
            
            ### ### ###
            
            return towel_req
        
        

In [15]:
"""
   Confirms Towels

"""


def towel_confirmation(towel_req):
   # VAR
    yes = '.*\\byes\\b.*|.*\\byep\\b.*|.*\\bcorrect\\b.*'
    no =  '.*\\bno\\b.*|.*\\bnegative\\b.*|.*\\bnope\\b.*|.*\\bnot\\b.*'
    
    vaReply = "to confirm with you, you would like '
    text_to_speech(vaReply)
    
    
    if towel_req[1][0] == 1:
        towel_req[1][1] 
        | towel_req[2][0] == 1 | towel_req[3][0] == 1: 
    
    while True:
        
        # input 
        human_input = speech_to_text()
        print('human input: ',bool(re.search(yes,human_input)))
        if re.search(no,human_input):
            vaReply = "no problem let me change that"
            text_to_speech(vaReply)
            return False, False
            
        elif re.search(yes,human_input):
            vaReply = "perfect"
            text_to_speech(vaReply)
            return True,True 
        else:
            vaReply = "I am sorry i did not understand that, you said room " + str(room_number) + ' is that correct?'
            text_to_speech(vaReply) 
        print('confirm loop')

In [15]:
"""
   Confirms Room numbers

"""


def room_confirmation(room_number):
   # VAR
    yes = '.*\\byes\\b.*|.*\\byep\\b.*|.*\\bcorrect\\b.*'
    no =  '.*\\bno\\b.*|.*\\bnegative\\b.*|.*\\bnope\\b.*|.*\\bnot\\b.*'
    
    vaReply = "you said room " + str(room_number) + ' is that correct?'
    text_to_speech(vaReply)
    
    while True:
        
        # input 
        human_input = speech_to_text()
        print('human input: ',bool(re.search(yes,human_input)))
        if re.search(no,human_input):
            vaReply = "no problem let me change that"
            text_to_speech(vaReply)
            return False, False
            
        elif re.search(yes,human_input):
            vaReply = "perfect"
            text_to_speech(vaReply)
            return True,True 
        else:
            vaReply = "I am sorry i did not understand that, you said room " + str(room_number) + ' is that correct?'
            text_to_speech(vaReply) 
        print('confirm loop')

In [16]:
"""
    Enters input for Room number

"""
def roomNumber():
   # VAR 
    flag = False
    keyRoomNumber = False
    clean_input = 0   

    vaReply = "What room number should I send this up too?"
    #text_to_speech(vaReply)
    
    while (keyRoomNumber == False):
        text_to_speech(vaReply)
       
        # input 
        human_input = speech_to_text()
        
        try:
            clean_input = int(re.sub('\D', '', human_input) )
            print('try - clean input: ',clean_input)
            
        except:
            vaReply = "I am sorry i did not get that, what is your room number?"
            text_to_speech(vaReply)
            print('except')
        
        if (clean_input <= 5000) & (clean_input > 0): 
            flag = True
            print('flag true')
            room_number = clean_input 
            
            
       # FLAG to confirm room number 
        if flag == True:
            keyRoomNumber , flag = room_confirmation(room_number)
    
    return room_number
            
            

In [17]:
if __name__ == "__main__":
    
    
    # flags
    flag = True
    new_topic = True

    
    # main loop 
    while (flag == True):
        # Welcome 
        if new_topic == True:
            text_to_speech('Hello this is the Tokyo Grand Hyatt, how can i help you')  
            new_topic = False
        elif new_topic == False:
            text_to_speech('Can we help you with anything else?')
            

            
        res = speech_to_text()

        if re.search(key_towel, res):
            towel_req = need_towels()
            text_to_speech('excelent, we will send them straight up!')
            print(towel_req)
            
        
            
            


Virtual Assistant:  Hello this is the Tokyo Grand Hyatt, how can i help you
listening now
Human: towels please
Virtual Assistant:  Can we help you with anything else?
listening now
Human: I need some towels please
Virtual Assistant:  what type of towels would you like: bath towels, hand towels, or washcloths
listening now
Human: bath towels please
Virtual Assistant:  how many bath towels would you like?
listening now
Human: 7
Virtual Assistant:  would you like any additional type of towel?
listening now
Human: no thank you
Virtual Assistant:  What room number should I send this up too?
listening now
Human: 623
try - clean input:  623
flag true
Virtual Assistant:  you said room 623 is that correct?
listening now
Human: yes
human input:  True
Virtual Assistant:  perfect
Virtual Assistant:  excelent, we will send them straight up!
[[623, 0], [1, 7], [0, 0], [0, 0]]
Virtual Assistant:  Can we help you with anything else?
listening now
Human: no thank you
Virtual Assistant:  Can we help you

KeyboardInterrupt: 